#### Qlunc tutorial by Francisco Costa
# **Tutorial 2**
## Calculate line of sight wind velocity ($V_{LOS}$) and horizontal wind velocity ($V_{h}$) uncertainties

This tutorial aims to facilitate the use to Qlunc. 
In this tutorial will deal with the new capabilities of Qlunc, namely $V_{LOS}$ (one single lidar) and $V_{h}$ (two lidars) estimations. To quantitatively verify the results a comparison between Montecarlo simulations and the analytical model (GUM approach) is performed.

* **Some steps are not deeply explained since it is not necessary for the sake of this tutorial. The main goal of the tutorial is to show how the lidar digital twin is built and the interaction between different modules.**

In [13]:
import os
os.chdir('../')
# importing  uncertainty functions
import Utils.Qlunc_Help_standAlone as SA, UQ_Functions.UQ_Photonics_Classes as uphc,UQ_Functions.UQ_Optics_Classes as uopc, UQ_Functions.UQ_Power_Classes as upwc,UQ_Functions.UQ_Lidar_Classes as ulc, UQ_Functions.UQ_ProbeVolume_Classes as upbc,UQ_Functions.UQ_Data_processing_Classes as uprm
from Utils.Qlunc_ImportModules import *

In [14]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 0C54-9785

 Directory of C:\

17.06.2021  10:43    <DIR>          .pytest_cache
08.12.2021  18:22    <DIR>          Anaconda3
11.02.2020  19:14    <DIR>          Intel
07.12.2019  10:14    <DIR>          PerfLogs
06.11.2022  08:25    <DIR>          Program Files
12.07.2022  04:00    <DIR>          Program Files (x86)
06.11.2022  19:21    <DIR>          SWE_LOCAL
07.06.2021  09:36    <DIR>          Users
29.10.2022  06:24    <DIR>          Windows
               0 File(s)              0 bytes
               9 Dir(s)  120.823.128.064 bytes free


A [.yml template](https://github.com/PacoCosta/Qlunc/blob/Qlunc-V0.9/Main/Template_yaml_inputs_file.yml) is included in the repository. This tutorial uses two different yaml files to create two lidars 'Caixa1' and 'Caixa2'. 

The code below imports the `yaml` file.

In [11]:
# Read yaml files 1 and 2
# Obtain data from .yaml file:
with open (r'./Tutorials/yaml_inputs_file_2_1.yml','r') as file:
    Qlunc_yaml_inputs={}
    docs = yaml.load_all(file, Loader=yaml.FullLoader)
    for doc in docs:      
        for k, v in doc.items():     
            Qlunc_yaml_inputs1.setdefault(k,v)
# Obtain data from .yaml file:
with open (r'./Tutorials/yaml_inputs_file_2_2.yml','r') as file:
    Qlunc_yaml_inputs={}
    docs = yaml.load_all(file, Loader=yaml.FullLoader)
    for doc in docs:      
        for k, v in doc.items():     
            Qlunc_yaml_inputs2.setdefault(k,v)

FileNotFoundError: [Errno 2] No such file or directory: './Tutorials/yaml_inputs_file_2_1.yml'